# Airline staffing problem


### Group: Augusto Tadashi MIYAGAWA and Eduardo DADALTO CAMARA GOMES


*Original lab session by Gérard Verfaillie.*

This problem is about the staffing of an airline, i.e. assigning pilots and crew members to scheduled flights using a Constraint Satisfaction/Optimisation Problem (CSP/COP) solver (here `facile`). Three instances of the problem are given.

As a pure CSP modelling fails for the larger instance, we will solve it using a combination of CSP and Mixed-Integer Linear Programming (MILP) modelling. You may refer to past classes with the `PuLP` library for basic usage.

<div class="alert alert-warning">
    <b>Warning</b>: You may submit in French or English. Make a choice and stick to it (don't mix languages).
</div>

In [1]:
import facile

## Reminders of syntax with the facile library

<div class="alert alert-success">
    <b>Hint</b>: Constraints may be summed: the result is the number of True values in the set of constraints.<br/>(However, this doesn't work with global constraints such as alldifferent)
</div>

Example: Find an array of length $n$ taking values in $[0, n-1]$ so that the $i$-th value is equal to the number of $i$ inside the array.

In [ ]:
n = 10
array = [facile.variable(range(n)) for i in range(n)]

for i in range(n):
    facile.constraint(sum([x == i for x in array]) == array[i])

if facile.solve(array):
    print(f"indices: {list(range(n))}")
    print(f"array:   {[v.value() for v in array]}")

<div class="alert alert-success">
    <b>Hint</b>: You may use the & and | operators for the logical $\land$ and $\lor$.<br/>(However, this doesn't work with global constraints such as alldifferent)
</div>

In [ ]:
a, b = facile.variable([0, 1, 2]), facile.variable([0, 1, 2])

left = (a == b) & (a + b == 2)
right = a > b
facile.constraint(left | right)
result = facile.solve([a, b])
a.value(), b.value()

<div class="alert alert-success">
    <b>Hint</b>: You may use the <code>solve_all</code> function to get all the solutions to a CSP problem.
</div>

In [ ]:
a, b = facile.variable([0, 1, 2]), facile.variable([0, 1, 2])

left = (a == b) & (a + b == 2)
right = a > b
facile.constraint(left | right)
result = facile.solve_all([a, b])

[x.solution for x in result]

<div class="alert alert-warning">
    <b>Warning</b>: Using <code>solve_all</code> does not assign any value to a variable. (The explanation lies in the fact the Branch&Bound algorithm returns with no solution found—hence the last <code>None</code>)<br/>Note the similar behaviour with the <code>minimize</code> function.
</div>

In [ ]:
a.value(), b.value()

<div class="alert alert-success">
    <b>Hint</b>: You may index a 1D-array of variables with a facile variable or expression. But you first need to wrap the array with the facile.array function.
</div>
<div class="alert alert-danger">
    <b>Warning</b>: Be aware that the indexing value will be constrained between 0 and max_index.
</div>
<div class="alert alert-warning">
    <b>Warning</b>: If you work with 2D-arrays, the linearisation of indices (remember your programming classes 😀) is left to you as part of the exercice. This should be something along a[i][j] becomes a[i*n + j]...
</div>

Example: find the position of the first 0 in a given array.

In [ ]:
a = facile.array([1, 2, 0, 4, 5, 0])
i = facile.variable(range(len(a)))
facile.constraint(a[i] == 0)
result = facile.minimize([i], i)
result.solution

## Problem description

An airline operates a given set of flights between pairs of airports. Each flight must be staffed with two pilots and enough cabin crew members depending on the type of aircraft.

Each staff member must start its trip from his hometown (not before 12am) and be back home in the evening (before 12am). They must not fly more than a maximum number of flights per day, and more than a maximum amount of time out of home (from first take-off time to last landing time in addition to commuting time).

Each staff member may fly as a passenger if they just need to be transferred to another airport/city. They may as well be assigned to just stay home.

We will first search for a solution to this problem, then find the optimal solution that will maximise the money made by the airline by minimising the number of passenger seats assigned to staff.

## Problem data

The following data are given:
- `Nv`: total number of flights;
- `Na`: total number of airports;
- `Ni`: total number of cities;
- `Va`: a table associating a city to an airport (they may be several airports in the same city);
- `Ov` (resp. `Dv`): this table associates an origin (resp. destination) airport to a flight;
- `Td` (resp. `Ta`): this table associates a take-off time (resp. landing time) to a flight, (in round hours);
- `Dt`: this table returns transit times between two airports of the same city (in hours). This time is equal to 25 (more than a day, so infinite) for airports in different cities; `Dt[i][i]` is the time required to transfer between two aircraft at airport $i$;
- `Np`: this table associates a number of passenger seats to a flight;
- `Nec`: this table associates a number of required cabin crew members to a flight;
- `Pr`: this table associates a price for a ticket in a flight;
- `Ne`: total number of staff in the airline;
- `Ty`: this table associates 1 to a pilot, 0 to a cabin crew member;
- `Vh`: this tables associates a hometown to each staff member;
- `Nvmax`: maximum number of flights per staff member per day;
- `Dmax`: maximum work time (out of home) per staff member per day;
- `Dda`: standard commuting time (both ways).

<div class="alert alert-success">
    <b>Hint</b>: Three instances are given (tiny, small, normal). Start debugging with the tiny instance; when it works, try the small instance: <b>you will be graded based on the results of the small instance</b>.
</div>
<div class="alert alert-warning">
    <b>Warning</b>: Keep the normal instance for next section. (Trying it with this modelling may stall the kernel and require a restart)
</div>

Index 0 in the tables printed below means "no flight", associated to "no airport" and "no city".

In [5]:
def load_airline(airline_size):
    if airline_size == 1:
        %run airline-tiny.py
    if airline_size == 2:
        %run airline-small.py
    if airline_size == 3:
        %run airline-normal.py

## CSP modelling

### Variable definitions

First, define the variables of the problem. You may associate a variable to each flight taken by a staff member on a given day.

#### Comments

<div class="alert alert-danger">
$schedule = \sum\limits_{i = 1}^{N_e}\sum\limits_{j = 0}^{N_{vmax}} var_{i,j}$

$schedule$ is a matrix of variables with $N_e$ rows and $N_{vmax}+1$ columns. The variable $var$ can have values from 0 to $N_v$    
</div>


In [6]:
load_airline(2)
schedule = [[facile.variable(0,Nv) for i in range(Nvmax+1)] for j in range(Ne)]

C:\Anaconda3\envs\tensorF\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


### Constraints definition (50%)

<div class="alert alert-warning">
    <b>Warning</b>: Please write and explain (use Python comments) the code for each constraint in the corresponding <b>code</b> cells.<br/>Use Python comments for explanations. <b>You will be graded accordingly.</b><br/>You may add as many cells as necessary but do not move existing cells.
</div>

- Flights assigned to `0` (no flight) must be at the end of the schedule.  
  `[1, 0, 2]` could be equivalent to `[1, 2, 0]` but only the second one is valid.

#### Comments

<div class="alert alert-danger">
Constraint :
    <br/>- The first column is equal to zero (because the matrix has $N_{vmax}+1$ columns).
    <br/>$schedule_{i,0} = 0$
    <br/>
    <br/>- The element of the matrix line is different to zero or the next element is equal to zero to guarantee that all flights assigned to 0 are in the end of the schedule.
    <br/> $schedule_{i,j} \neq 0$ | $schedule_{i,j+1} = 0$ ($schedule_{i,j} = 0$ $\rightarrow$ $schedule_{i,j+1} = 0$)
</div>

In [9]:
for line in schedule:
    facile.constraint(line[0] == 0)
    for j in range(1, Nvmax):
        
        # Constraint
        facile.constraint((line[j] != 0) | (line[j+1] == 0))

- If a staff member flies on that day, then (s)he leaves home and returns home.

#### Comments

<div class="alert alert-danger">
    Constraint :
    <br/> - City of departure is the same as staff member hometown and  is the first flight ($facile.constraint( (citydeparture == hometown) | (flight1 == 0))$)
    <br/> $citydeparture \neq hometow \rightarrow flight1 = 0$
    <br/>
    <br/> - The last flight arrives in the hometown ($facile.constraint((Va[Dv[lastflight]] == hometown) | (flight1 == 0))$)
    <br/> $Va[Dv[lastflight]] \neq hometow \rightarrow flight1 = 0$
</div>

In [10]:
# Transform the variables in a facile.array
Ov = facile.array(Ov)
Dv = facile.array(Dv)
Va = facile.array(Va)
Ta = facile.array(Ta)
Td = facile.array(Td)

vide = []
for j in Dt:
    for i in j:    
        vide.append(i)
Dt2 = facile.array(vide)

# Define constraints
for i in range(Ne):
    flight1 = schedule[i][1]
    hometown = Vh[i]
    citydeparture = Va[Ov[flight1]]
    
    # Constraint
    facile.constraint( (citydeparture == hometown) | (flight1 == 0))
    
    staff = schedule[i][:]
    last = sum([staff[i]!=0 for i in range(Nvmax+1)])
    staff_array = facile.array(staff)
    lastflight = staff_array[last]
    destination_last = Va[Dv[lastflight]]
    
    # Constraint
    facile.constraint((Va[Dv[lastflight]] == hometown) | (flight1 == 0))

- A staff member may chain two flights iff :
    - first flight's destination airport and second flight's origin airport cities match;
    - there is enough transfer time between flights.

#### Comments

<div class="alert alert-danger">
    Constraint :
    <br/> - First flight's destination airport ($schedule_{i,j}$) and second flight's origin airport ($schedule_{i,j+1}$) cities match ($facile.constraint((fist \: destination == second\:origin) | (second\:flight == 0))$);
    <br/> - The conexion time is longer than transit time ($facile.constraint((conexion\:time>=transit\:time) | (second\:flight == 0))$)
</div>

In [11]:
for i in range(Ne):
    staff = schedule[i][:]
    for j in range(1,Nvmax):
        first_flight = staff[j]
        second_flight = staff[j+1]
        
        fist_destination = Va[Dv[first_flight]]
        second_origin = Va[Ov[second_flight]]
        
        # Constraint
        facile.constraint((fist_destination == second_origin) | (second_flight == 0))
        
        first_landing = Ta[first_flight]
        second_takeoff = Td[second_flight]
        conexion_time = second_takeoff - first_landing
        first_airport = Dv[first_flight]
        second_airport = Ov[second_flight]
        transit_time = Dt2[first_airport*(Na+1) + second_airport]
        
        # Constraint
        facile.constraint((conexion_time>=transit_time) | (second_flight == 0))
   

- A staff member may not exceed his/her total time out of home;

#### Comments

<div class="alert alert-danger">
    Constraint :
    <br/> - The time between flights (time) plus standard commuting time (Dda) must be smaller than maximum work time per staff member (Dmax) <br/>($facile.constraint(((time + Dda) <= (Dmax)))$). 
</div>

In [12]:
for i in range(Ne):
    flight1 = schedule[i][1]
    staff = schedule[i][:]
    last = sum([staff[j]!=0 for j in range(Nvmax+1)])
    staff_array = facile.array(staff)
    lastflight = staff_array[last]
    time = Ta[lastflight]-Td[flight1]
    
    # Constraint
    facile.constraint(((time + Dda) <= (Dmax)))

- Each flights requires at least two pilots and enough cabin crew members.

#### Comments

<div class="alert alert-danger">
    Constraint :
    <br/> - The crew must be bigger or equal to than the minimum required in a cabin during a flight ($facile.constraint(crews >= Nec[flight])$).
    <br/> - Requires at least two pilots ($facile.constraint(pilots >= 2)$).
</div>

In [13]:
vide = []
for j in schedule:
    for i in j:    
        vide.append(i)
schedule2 = facile.array(vide)
for flight in range(1,Nv+1):
    crews = sum([schedule2[i*(Nvmax+1) + j] == flight for i in range(Ne) for j in range(Nvmax+1) if Ty[i] == 0])
    pilots = sum([schedule2[i*(Nvmax+1) + j] == flight for i in range(Ne) for j in range(Nvmax+1) if Ty[i] == 1])
    
    # Constraint
    facile.constraint(crews >= Nec[flight])
    
    # Constraint
    facile.constraint(pilots >= 2)
    

### Problem optimisation (5%)

In [14]:
# maximize revenue
revenue = facile.variable(0, 1000000)
for flight in range(1,Nv+1):
    crews = sum([schedule2[i*(Nvmax+1) + j] == flight for i in range(Ne) for j in range(Nvmax+1) if Ty[i] == 0])
    pilots = sum([schedule2[i*(Nvmax+1) + j] == flight for i in range(Ne) for j in range(Nvmax+1) if Ty[i] == 1])

    seated = (Np[flight] - ((pilots-2) + (crews - Nec[flight])))
    revenue = revenue + seated*Pr[flight]
result = facile.minimize(schedule2, -revenue)

### Answer the problem (10%)

<div class='alert alert-warning'>
To maximise your grade, please post-process and pretty-print the results of the optimisation and check they are consistent with the constraints. <b>If you find inconsistencies you cannot solve, please point them.</b> (You will fail this question if you pretend everything is fine or if you try to hide your issues)
</div>

For each staff member, list the flights (s)he will be flying with cities and take-off/landing times.

Valid example:
```
Staff number 0: 1 [1, 2] 2 [2, 1] scheduled at [8, 9] [12, 13]
Staff number 1: ...
```
    
For each flight, list the pilots and cabin crew members flying (together with the required staffing of the aircraft).

Valid example:
```
Flight number 1: pilots {0, 1} cabin crew {2, 3} (≥ 2) ;
Flight number 2: ...
```


In [19]:
for i in range(Ne):
    vec = result.solution[i*(Nvmax+1)+1:i*(Nvmax+1)+(Nvmax+1)]
    print('\nStaff number {}: '.format(i), end='')
    [print('flight {} from/to [{}, {}] schedule at [{}, {}], '
           .format(j, Va[Ov[j]].value(),Va[Dv[j]].value(), Ta[j].value(), Td[j].value()),  end='') for j in vec if j >0] 

for flight in range(1,Nv+1):
    print('\nFlight number {}: '.format(flight), end='')
    pilots = set()
    crews = set()
    for staff in range(Ne):
        vec = result.solution[staff*(Nvmax+1)+1:staff*(Nvmax+1)+(Nvmax+1)]
        if flight in vec:
            if Ty[staff]:
                pilots.add(staff) 
            else:
                crews.add(staff)
    num = Nec[flight]
    [print('pilots {} cabin crew {} (≥ {})'.format(pilots, crews, num), end='')]



Staff number 0: flight 1 from/to [1, 2] schedule at [8, 7], flight 4 from/to [2, 4] schedule at [13, 12], flight 6 from/to [4, 1] schedule at [20, 18], 
Staff number 1: flight 1 from/to [1, 2] schedule at [8, 7], flight 4 from/to [2, 4] schedule at [13, 12], flight 6 from/to [4, 1] schedule at [20, 18], 
Staff number 2: flight 2 from/to [1, 3] schedule at [9, 8], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 3: flight 2 from/to [1, 3] schedule at [9, 8], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 4: flight 3 from/to [1, 4] schedule at [11, 9], flight 7 from/to [4, 3] schedule at [19, 18], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 5: flight 3 from/to [1, 4] schedule at [11, 9], flight 7 from/to [4, 3] schedule at [19, 18], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 6: flight 1 from/to [1, 2] schedule at [8, 7], flight 4 from/to [2, 4] schedule at [13, 12], flight 6 from/to [4, 1] schedule at [20, 18], 
Staff number 7: fl

## A different modelling for CSP and MILP

Since bigger instances will require too much computing time, let's try something smarter:
1. First use a CSP solver to compute all possible paths between cities;
2. Then compare performances between a CSP and a MILP solver as we no longer assign staff members but groups of staff members to each path.

<div class="alert alert-success">
    <b>Hint</b>: Three instances are given (tiny, small, normal). Start debugging with the small instance; when it works, try the normal instance: you will be graded based on the results of the <b>normal instance</b>.
</div>

### First questions (10%)

<div class="alert alert-warning">
    <b>Warning</b>: Do not expect any external assistance for these questions. You are on your own.
</div>

1. Why would it help to solve the problem for groups of staff member rather than individuals. Which mathematical principle enters into account?
2. Try to explain why CSP better suits (compared to MILP) the path computing part of the problem.

<div class="alert alert-info">
    <b>Answer question 1</b>: 
    <br\>When we group the staff, the search tree shrinks and avoid the combinatorial explosion.
</div>

<div class="alert alert-info">
    <b>Answer question 2</b>: 
    <br\>The CSP finds all possible solutions to the problem, while the MILP optimize the solution search to find the optimal one. Thus, the MILP does not check every possible solution or path.
</div>

### Compute all possible paths (10%)

In [20]:
load_airline(2)

<div class="alert alert-danger">
    <b>Transformation of variable types</b> 
</div>

In [21]:
Ov = facile.array(Ov)
Dv = facile.array(Dv)
Va = facile.array(Va)
Ta = facile.array(Ta)
Td = facile.array(Td)

vide = []
for j in Dt:
    for i in j:    
        vide.append(i)
Dt2 = facile.array(vide)

<div class="alert alert-danger">
    <b>Decision variable</b> 
</div>

In [22]:
paths = [facile.variable(0,Nv) for i in range(Nvmax+1)]

<div class="alert alert-danger">
    <b>First element is zero for indexing</b> 
</div>

In [23]:
facile.constraint(paths[0] == 0)

<div class="alert alert-danger">
    <b>Zeros in the end of each line of the matrix</b> 
</div>

In [24]:
for j in range(1, Nvmax):
    facile.constraint((paths[j] != 0) | (paths[j+1] == 0))

<div class="alert alert-danger">
    <b>Last city equals the first city</b> 
</div>

In [25]:
first_flight = paths[1]
paths_array = facile.array(paths)
last = sum(paths[j]!=0 for j in range(Nvmax+1))
last_flight = paths_array[last]

facile.constraint(Va[Ov[first_flight]] == Va[Dv[last_flight]])

<div class="alert alert-danger">
    <b>Constraint :</b>
    <br\>- All flights connected
    <br\>- Enough connection time between flights
</div>

In [26]:


for j in range(1,Nvmax):
    first_flight = paths[j]
    second_flight = paths[j+1]
        
    first_destination = Va[Dv[first_flight]]
    second_origin = Va[Ov[second_flight]]
    
    # Constraint
    facile.constraint((first_destination == second_origin) | (second_flight == 0))
        
    first_landing = Ta[first_flight]
    second_takeoff = Td[second_flight]
    conexion_time = second_takeoff - first_landing
    first_airport = Dv[first_flight]
    second_airport = Ov[second_flight]
    transit_time = Dt2[first_airport*(Na+1) + second_airport]
    
    # Constraint
    facile.constraint((conexion_time>=transit_time) | (second_flight == 0))
        

<div class="alert alert-danger">
    <b>Constraint :</b>
    <br\>- Total path time does not exceed maximum work time
</div>

In [27]:
facile.constraint(Ta[last_flight]-Td[first_flight] + Dda <= Dmax)

<div class="alert alert-danger">
    <b>Solution computation</b> 
</div>

In [28]:
result = facile.solve_all(paths)

all_paths = [x.solution for x in result][:-1]
print('All possible paths are: {}'.format(all_paths))

All possible paths are: [[0, 0, 0, 0], [0, 1, 4, 6], [0, 2, 5, 0], [0, 3, 6, 0], [0, 3, 7, 5]]


### CSP and MILP resolution (15%)

- Assign a group of staff members to each path;
- Compare the computing time with CSP and MILP resolutions (use `PuLP` library for MILP);
- Post-process and pretty-print the results;
- Comment

<div class="alert alert-danger">
    <b>Supplementary variable</b>
    <br\>- For each path there is an affected group of pilot + crew
</div>

In [29]:
all_paths = [x.solution for x in result][:-1]
num_paths = len(all_paths)
staffs = [facile.variable(0,num_paths-1) for j in range(Ne)]

vide = []
for j in all_paths:
    for i in j:    
        vide.append(i)
all_paths2 = facile.array(vide)

<div class="alert alert-danger">
    <b>Path's origin city equals the hometown of each staff from group G</b> 
</div>

In [30]:
for i in range(Ne):
    G = staffs[i]
    first_flight = all_paths2[G*(Nvmax+1) + 1]
    origin_city = Va[Ov[first_flight]]
    hometown = Vh[i]
    facile.constraint((hometown == origin_city) | (first_flight==0))

<div class="alert alert-danger">
    <b>Check if every flight is in the solution</b> 
</div>

In [31]:
for flight in range(1, Nv+1):
    flight_occurence = 0
    for k in range(Ne):
        flight_occurence += sum([all_paths2[staffs[k]*(Nvmax+1) + j] == flight
                            for j in range(1,Nvmax+1)])
    facile.constraint(flight_occurence >= 1)

<div class="alert alert-danger">
    <b>Check if there is enough pilots and crew per flight</b> 
</div>

In [32]:
for flight in range(1,Nv+1):
    num_crew = Nec[flight]
    num_pilot= 2
    crew_presence = 0
    pilot_presence = 0
    for k in range(Ne):
        crew_presence += sum([(all_paths2[staffs[k]*(Nvmax+1) + j] == flight)*(1-Ty[k])
                            for j in range(1,Nvmax+1)])
        pilot_presence += sum([(all_paths2[staffs[k]*(Nvmax+1) + j] == flight)*Ty[k]
                            for j in range(1,Nvmax+1)])
    facile.constraint(crew_presence >= num_crew)
    facile.constraint(pilot_presence >= num_pilot)

<div class="alert alert-danger">
    <b>Revenue maximization</b> 
</div>

In [33]:
revenue = facile.variable(0, 1000000)
for flight in range(1,Nv+1):
    crews = 0
    pilots = 0
    for k in range(Ne):
        crews += sum([(all_paths2[staffs[k]*(Nvmax+1) + j] == flight)*(1-Ty[k])
                            for j in range(1,Nvmax+1)])
        pilots += sum([(all_paths2[staffs[k]*(Nvmax+1) + j] == flight)*Ty[k]
                            for j in range(1,Nvmax+1)])

    seated = (Np[flight] - ((pilots-2) + (crews - Nec[flight])))
    revenue += seated*Pr[flight]

In [34]:
afect = facile.minimize(staffs, -revenue)

<div class="alert alert-danger">
    <b>Pretty print</b> 
</div>

In [35]:
schedule_made = []
for i in afect.solution:
    schedule_made.append(all_paths[i])

for i in range(Ne):
    vec = schedule_made[i][1:]
    print('\nStaff number {}: '.format(i), end='')
    [print('flight {} from/to [{}, {}] schedule at [{}, {}], '
           .format(j, Va[Ov[j]].value(),Va[Dv[j]].value(), Ta[j].value(), Td[j].value()),  end='') for j in vec if j >0] 

for flight in range(1,Nv+1):
    print('\nFlight number {}: '.format(flight), end='')
    pilots = set()
    crews = set()
    for staff in range(Ne):
        vec = schedule_made[staff][1:]
        if flight in vec:
            if Ty[staff]:
                pilots.add(staff) 
            else:
                crews.add(staff)
    num = Nec[flight]
    [print('pilots {} cabin crew {} (≥ {})'.format(pilots, crews, num), end='')]



Staff number 0: flight 1 from/to [1, 2] schedule at [8, 7], flight 4 from/to [2, 4] schedule at [13, 12], flight 6 from/to [4, 1] schedule at [20, 18], 
Staff number 1: flight 1 from/to [1, 2] schedule at [8, 7], flight 4 from/to [2, 4] schedule at [13, 12], flight 6 from/to [4, 1] schedule at [20, 18], 
Staff number 2: flight 2 from/to [1, 3] schedule at [9, 8], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 3: flight 2 from/to [1, 3] schedule at [9, 8], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 4: flight 3 from/to [1, 4] schedule at [11, 9], flight 7 from/to [4, 3] schedule at [19, 18], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 5: flight 3 from/to [1, 4] schedule at [11, 9], flight 7 from/to [4, 3] schedule at [19, 18], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 6: flight 1 from/to [1, 2] schedule at [8, 7], flight 4 from/to [2, 4] schedule at [13, 12], flight 6 from/to [4, 1] schedule at [20, 18], 
Staff number 7: fl

<div class="alert alert-info">
    camiho -> espaço de pesquisa
</div>

- Assign a group of staff members to each path;
- Compare the computing time with CSP and MILP resolutions (use `PuLP` library for MILP);
- Post-process and pretty-print the results;
- Comment

# MILP using PuLP

In [36]:
from pulp import *

In [37]:
load_airline(2)

<div class="alert alert-danger">
    <b>Create the 'prob' variable to contain the problem data</b> 
</div>

In [38]:
prob = LpProblem("The Afectation Problem", LpMaximize)

<div class="alert alert-danger">
    <b>Solution space</b> 
</div>

In [39]:
all_paths = [x.solution for x in result][:-1]
num_paths = len(all_paths)
# Match staff and path
matches = [tuple([staff,g]) for staff in range(Ne) 
                for g in range(num_paths)
                if ((Vh[staff] == Va[Ov[all_paths[g][1]]])|(all_paths[g][1] == 0))] # check staff hometown
                  
x = pulp.LpVariable.dicts('', matches, 
                            lowBound = 0,
                            upBound = 1,
                            cat = LpInteger)

<div class="alert alert-danger">
    <b>Objective function</b> 
</div>

In [40]:
revenue = 0
for staff, index_path in matches:
    crews = 0
    pilots = 0
    if x[tuple([staff, index_path])] != 0:
        for flight in range(1, Nv+1):
            crews += sum([(all_paths[index_path][j] == flight)*(1-Ty[staff])
                            for j in range(1,Nvmax+1)])
            pilots += sum([(all_paths[index_path][j] == flight)*Ty[staff]
                            for j in range(1,Nvmax+1)])
            
            seated = (Np[flight] - ((pilots-2) + (crews - Nec[flight])))
            revenue += seated*Pr[flight]*x[tuple([staff, index_path])]
    
prob += revenue

<div class="alert alert-danger">
    <b>Check if each staff is in a group</b> 
</div>

In [41]:
for staff in range(Ne):
    prob += sum([x[key] for key in matches if (key[0] == staff)]) == 1

<div class="alert alert-danger">
    <b>Check if each flight is in the paths</b> 
</div>

In [42]:
for flight in range(1, Nv+1):
    prob += sum([x[tuple([staff, g])] for staff,g in matches
                                if flight in all_paths[g]]) >= 1
    
    # Check minimum number of pilots and crew member 
    # pilots
    prob += sum([Ty[staff]*x[tuple([staff, index_path])] for staff,index_path in matches
                    if flight in all_paths[index_path]]) >= 2
    # crew members
    prob += sum([(1-Ty[staff])*x[tuple([staff, index_path])] for staff,index_path in matches
                    if flight in all_paths[index_path]]) >= Nec[flight]

<div class="alert alert-danger">
    <b>Solve problem</b> 
</div>

In [43]:
resultPulp = prob.solve()

<div class="alert alert-danger">
    <b>Print solution</b> 
</div>

In [45]:
schedule_made = []
for key in matches:
    if x[key].value() == 1.0:
        schedule_made.append(all_paths[key[1]])

for i in range(Ne):
    vec = schedule_made[i][1:]
    print('\nStaff number {}: '.format(i), end='')
    [print('flight {} from/to [{}, {}] schedule at [{}, {}], '
           .format(j, Va[Ov[j]],Va[Dv[j]], Ta[j], Td[j]),  end='') for j in vec if j>0] 

for flight in range(1,Nv+1):
    print('\nFlight number {}: '.format(flight), end='')
    pilots = set()
    crews = set()
    for staff in range(Ne):
        vec = schedule_made[staff][1:]
        if flight in vec:
            if Ty[staff]:
                pilots.add(staff) 
            else:
                crews.add(staff)
    num = Nec[flight]
    [print('pilots {} cabin crew {} (≥ {})'.format(pilots, crews, num), end='')]


Staff number 0: flight 1 from/to [1, 2] schedule at [8, 7], flight 4 from/to [2, 4] schedule at [13, 12], flight 6 from/to [4, 1] schedule at [20, 18], 
Staff number 1: flight 1 from/to [1, 2] schedule at [8, 7], flight 4 from/to [2, 4] schedule at [13, 12], flight 6 from/to [4, 1] schedule at [20, 18], 
Staff number 2: flight 3 from/to [1, 4] schedule at [11, 9], flight 7 from/to [4, 3] schedule at [19, 18], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 3: flight 2 from/to [1, 3] schedule at [9, 8], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 4: flight 3 from/to [1, 4] schedule at [11, 9], flight 7 from/to [4, 3] schedule at [19, 18], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 5: flight 2 from/to [1, 3] schedule at [9, 8], flight 5 from/to [3, 1] schedule at [24, 23], 
Staff number 6: flight 3 from/to [1, 4] schedule at [11, 9], flight 6 from/to [4, 1] schedule at [20, 18], 
Staff number 7: flight 2 from/to [1, 3] schedule at [9, 8], fli